In [32]:
# Load libraries
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report,  roc_auc_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import collections

In [45]:
# Import text data
raw_training = pd.read_csv("training.csv")
raw_testing = pd.read_csv("test.csv")

# Create bag of words
#count = CountVectorizer()
#X_valid = count.fit_transform(raw_testing["article_words"])

# testing y
y_valid = raw_testing["topic"]

# Create bag of words
count = CountVectorizer()
bag_of_words = count.fit_transform(raw_training["article_words"])

# Create feature matrix
X = bag_of_words

# Create bag of words
y = raw_training["topic"]

#transform testing data
X_valid = count.transform(raw_testing["article_words"])

#######################Resampling Dataset#######################


# Reducing the effect imbalnced by deleting some irrelevant class
# "Irrelevant" classe has 4734 samples in the training data, try to reduce it into 2000
irrelevant = raw_training[raw_training["topic"] == "IRRELEVANT"]
remove_n = 2734
drop_indices = np.random.choice(irrelevant.index, remove_n, replace=False)
irrelevant = irrelevant.drop(drop_indices)

reduce_training =  pd.concat([raw_training[raw_training["topic"] != "IRRELEVANT"], irrelevant],ignore_index=True)
reduce_bag_of_words = count.fit_transform(reduce_training["article_words"])
R_X = reduce_bag_of_words
R_y = reduce_training["topic"]
#transform testing data
R_X_valid = count.transform(reduce_training["topic"])

# Icreasing the minor classes
# Increasing 
topic_class = raw_training[raw_training["topic"] != "IRRELEVANT"]
increase_training = pd.concat([topic_class, topic_class, topic_class, raw_training[raw_training["topic"] == "IRRELEVANT"]], ignore_index=True)
increase_bag_of_words = count.fit_transform(increase_training["article_words"])
I_X = increase_bag_of_words
I_y = increase_training["topic"]
#transform testing data
I_X_valid = count.transform(increase_training["topic"])

# Considering stop words
count2 = CountVectorizer(stop_words='english')
sw_bag_of_words = count.fit_transform(increase_training["article_words"])
s_X = sw_bag_of_words
s_y = increase_training["topic"]
#transform testing data
s_X_valid = count.transform(increase_training["topic"])

In [36]:
I_X

<19032x35822 sparse matrix of type '<class 'numpy.int64'>'
	with 1542963 stored elements in Compressed Sparse Row format>

In [46]:
# Using specific model "method", return specifc score "score" by cross validation

def Model_Score (X, y, x_valid, method):
   
    clf = method
    clf.fit(X, y)
    
    y_predicted = clf.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_predicted)
    
    precision = precision_score(y_valid, y_predicted, average = "macro")
    recall = recall_score(y_valid, y_predicted, average = "macro")
    f1 = f1_score(y_valid, y_predicted, average = "macro")
    
    return accuracy, precision, recall, f1

In [47]:
raw_accuracy, raw_precision, raw_recall, raw_f1 = Model_Score(X, y, X_valid, DecisionTreeClassifier())

# reduce irrelevant samples
R_accuracy, R_precision, R_recall, R_f1 = Model_Score(R_X, R_y, R_X_valid, DecisionTreeClassifier())

# Icreasing the minor classes
I_accuracy, I_precision, I_recall, I_f1 = Model_Score(I_X, I_y, I_X_valid, DecisionTreeClassifier())

# Try considering stop words
s_accuracy, s_precision, s_recall, s_f1 = Model_Score(s_X, s_y, s_X_valid, DecisionTreeClassifier())

ValueError: Number of features of the model must match the input. Model n_features is 29824 and input n_features is 35822 

In [43]:
X_train, X_valid, y_train, y_valid = train_test_split(s_X,s_y, test_size=0.20, random_state=42)
clf = DecisionTreeClassifier()

clf = clf.fit(X_train,y_train)

y_predicted = clf.predict(X_valid)

#print("Accuracy:",accuracy_score(y_valid, y_predicted))
#print("Precision:",precision_score(y_valid, y_predicted, average = "macro"))
#print("Recall:",recall_score(y_valid, y_predicted, average = "macro"))
#print("f1:",f1_score(y_valid, y_predicted, average = "macro"))

In [50]:
print("====================================================================================")

print("Decsision Tree using raw data:\naccuracy:  " + str(raw_accuracy) +
     "\nprecision: " + str(raw_precision) + "\nrecall:    " + str(raw_recall) + "\nf1:        " +
     str(raw_f1))
print("====================================================================================")

print("Decision Tree irrelevant classes:\naccuracy:  " + str(R_accuracy) +
     "\nprecision: " + str(R_precision) + "\nrecall:    " + str(R_recall) + "\nf1:        " +
     str(R_f1))
print("====================================================================================")

print("Decision Tree increasing minor classes:\naccuracy:  " + str(I_accuracy) +
     "\nprecision: " + str(I_precision) + "\nrecall:    " + str(I_recall) + "\nf1:        " +
     str(I_f1))
print("====================================================================================")

print("Decision Tree using minorcalsses with stop words:\naccuracy:  " + str(s_accuracy) +
     "\nprecision: " + str(s_precision) + "\nrecall:    " + str(s_recall) + "\nf1:        " +
     str(s_f1))

Decsision Tree using raw data:
accuracy:  0.714
precision: 0.5117374545940743
recall:    0.5203505590519321
f1:        0.4987327609084799
Decision Tree irrelevant classes:
accuracy:  0.6536189069423929
precision: 0.5517419931968679
recall:    0.5763876602547499
f1:        0.5601164681260884
Decision Tree increasing minor classes:
accuracy:  0.9130548988705017
precision: 0.8953737156383645
recall:    0.9491920688247649
f1:        0.919409961592923
Decision Tree using minorcalsses with stop words:
accuracy:  0.9072760703966378
precision: 0.8795603024538944
recall:    0.9464466679184834
f1:        0.9088671887099093


In [ ]:
len(y_valid)